In [390]:
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display as ipd
from pandas import read_excel
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import keras
from keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
metaData = read_excel(r'C:/Users/aanal/Documents/AML-Lab/VSD.xlsx', sheet_name='read_dataset')

In [3]:
metaData.head(10)

,File_segment_name,Duration,Violence_start,Violence_end,Violence_duration
0,angry_011,117.0,21.0,38.000,17.00
1,angry_011,117.0,40.0,55.000,15.00
2,angry_011,117.0,60.0,79.000,19.00
3,angry_011,117.0,85.0,95.000,10.00
4,angry_011,117.0,101.0,110.000,9.00
5,angry_012,32.0,0.0,0.020,0.02
6,angry_013,72.0,0.0,0.010,0.00
7,angry_014,50.0,0.0,0.012,0.00
8,angry_015,162.0,64.0,71.000,7.00
9,angry_015,162.0,89.0,100.000,11.00


In [4]:
metaData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   File_segment_name  341 non-null    object 
 1   Duration           341 non-null    float64
 2   Violence_start     341 non-null    float64
 3   Violence_end       341 non-null    float64
 4   Violence_duration  341 non-null    float64
dtypes: float64(4), object(1)
memory usage: 13.4+ KB


In [5]:
metaData['Duration'].describe()

count     341.000000
mean      173.798208
std       483.313948
min        15.000000
25%        74.000000
50%       108.000000
75%       161.000000
max      4966.000000
Name: Duration, dtype: float64

In [6]:
metaData['Violence_duration'].describe()

count    341.000000
mean       8.946962
std       12.588253
min        0.000000
25%        2.508000
50%        4.644000
75%       10.053000
max      141.260000
Name: Violence_duration, dtype: float64

In [77]:
audioDataPath = 'audios_VSD'
# C:/Users/aanal/Documents/AML-Lab/

In [8]:
audioFiles = os.listdir(audioDataPath)
len(audioFiles)

131

In [9]:
def extract_features(audio_path, start_time, end_time, sr=22050,  n_fft=1024):
    # Load the entire audio file
    y, sr = librosa.load(audio_path, sr=sr)
    
    # Convert start and end times to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)
    
    # Extract the segment of interest
    segment = y[start_idx:end_idx]
    
    # Compute MFCC features for the segment
    mfcc_segment = librosa.feature.mfcc(y=segment, sr=sr, n_fft=n_fft) #, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)
    
    # Compute MFCC features for the rest of the audio
    rest_audio = np.concatenate([y[:start_idx], y[end_idx:]])
    mfcc_rest = librosa.feature.mfcc(y=rest_audio, sr=sr, n_fft=n_fft) #, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)
    
    return mfcc_segment, mfcc_rest




In [ ]:
def fun(df, filepath):
    lst = []
    name = df['File_segment_name'][0]
    start = 0
    for i in range(len(df)):
        if name != df['File_segment_name'][i]:
            audioPath = os.path.join(filepath, df['File_segment_name'][i-1] + '.wav')
            a = extract_features(audioPath, df['Violence_end'][i-1], df['Duration'][i-1])
            start = 0
            name = df['File_segment_name'][i]
            lst.append([a,0])
        audioPath = os.path.join(filepath, df['File_segment_name'][i] + '.wav')
        nv = extract_features(audioPath, start, df['Violence_start'][i]) 
        lst.append([nv,0])
        v = extract_features(audioPath, df['Violence_start'][i], df['Violence_end'][i]) 
        lst.append([v,1])
        start = df['Violence_end'][i]
    audioPath = os.path.join(filepath,name + '.wav')
    nv = extract_features(audioPath, start, metaData.iloc[-1, 1]) 
    lst.append([nv,0])    
    return pd.DataFrame(lst, columns=['feature', 'lable'])

In [115]:
newData = fun(metaData[:10],audioDataPath)

C:\Users\aanal\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=0
  warnings.warn(
C:\Users\aanal\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=441
  warnings.warn(
C:\Users\aanal\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=220
  warnings.warn(
C:\Users\aanal\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=264
  warnings.warn(


In [116]:
newData

,feature,lable
0,"([[-283.0617, -299.2603, -314.5908, -253.0902,...",0
1,"([[-336.70837, -320.17462, -325.77048, -331.77...",1
2,"([[-297.0687, -253.70218, -228.25157, -226.497...",0
3,"([[-141.14969, -133.57347, -134.88489, -141.36...",1
4,"([[-185.53253, -198.32878, -209.37585, -218.36...",0
5,"([[-293.8736, -232.80789, -209.51224, -174.620...",1
6,"([[-335.6747, -263.656, -210.67967, -192.7073,...",0
7,"([[-84.04805, -122.31657, -115.65719, -126.689...",1
8,"([[-39.012417, -94.36844, -117.69629, -124.144...",0
9,"([[-308.69864, -271.56873, -235.51297, -234.77...",1


In [392]:
y = np.array(newData['lable']).reshape((-1, 1))
y = np.asarray(y).astype(np.float32)

In [393]:
X = np.array(newData['feature'])
X = X.reshape((-1, 1))
#y_list = newData['lable']
#y = np.array(y_list).reshape((-1, 1))
X.shape, y.shape

((25, 1), (25, 1))

In [394]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [395]:
X_test.shape, y_test.shape, X_train.shape, y_train.shape

((5, 1), (5, 1), (20, 1), (20, 1))

In [396]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 128)               256       
                                                                 
 dense_15 (Dense)            (None, 64)                8256      
                                                                 
 dense_16 (Dense)            (None, 1)                 65        
                                                                 
Total params: 8577 (33.50 KB)
Trainable params: 8577 (33.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [397]:

# Define your sequential model
model = Sequential()

# Add layers to your model
model.add(Dense(128, activation='relu', input_shape=(1,)))  # Adjust input shape
model.add(Dense(64, activation='relu'))   # Add another dense layer
model.add(Dense(1, activation='sigmoid'))  # Add the output layer with sigmoid activation for binary classification

# Compile your model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train your model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 0.4500 - val_loss: 0.0000e+00 - val_accuracy: 0.2000
Epoch 2/10
1/1 [==============================] - 0s 75ms/step - loss: 0.0000e+00 - accuracy: 0.4500 - val_loss: 0.0000e+00 - val_accuracy: 0.2000
Epoch 3/10
1/1 [==============================] - 0s 66ms/step - loss: 0.0000e+00 - accuracy: 0.4500 - val_loss: 0.0000e+00 - val_accuracy: 0.2000
Epoch 4/10
1/1 [==============================] - 0s 68ms/step - loss: 0.0000e+00 - accuracy: 0.4500 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 5/10
1/1 [==============================] - 0s 82ms/step - loss: 0.0000e+00 - accuracy: 0.5500 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 6/10
1/1 [==============================] - 0s 66ms/step - loss: 0.0000e+00 - accuracy: 0.5500 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 7/10
1/1 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - accuracy: 0.5500 - val_loss: 0.0000e

In [398]:
AcuuracyTest = model.evaluate(X_test, y_test, verbose=0)

In [401]:
AcuuracyTest[1] * 100

80.0000011920929